In [17]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# 파일 불러오기
info_df = pd.read_csv("통합_정보.csv", encoding='euc-kr')
combined_df = pd.read_csv("C:/Users/songbh0304/OneDrive - 숭실대학교 - Soongsil University/바탕 화면/데이터톤/borrowclub/통합_정보_합침.csv", encoding='euc-kr')

# 중복책권수 데이터 변환 및 확인
combined_df['중복책권수'] = pd.to_numeric(combined_df['중복책권수'].replace(',', '', regex=True), errors='coerce').fillna(0).astype(int)

# 변환 후 중복책권수 확인
print("변환된 중복책권수 값 확인:")
print(combined_df[['도서ID', '중복책권수']].head(10))

# 1. '마지막 대출일자'와 '총 대출수' 기준 점수 계산
info_df['마지막 대출일자'] = pd.to_datetime(info_df['마지막대출일자'], errors='coerce')
min_date = info_df['마지막 대출일자'].min()

def last_loan_priority(date):
    if pd.isna(date):
        return 1  # '대출 기록 없음'에 가장 높은 점수 부여
    return (date - min_date).days  # 오래된 대출일수록 낮은 점수

info_df['대출일자_점수'] = info_df['마지막 대출일자'].apply(last_loan_priority)
info_df['대출일자_점수'] = MinMaxScaler().fit_transform(info_df[['대출일자_점수']])  # 정규화

# '총 대출수' 처리 (대출 수가 적을수록 높은 점수 부여)
info_df['총대출수'] = pd.to_numeric(info_df['총대출수'], errors='coerce').fillna(0)
info_df['대출수_점수'] = 1 - MinMaxScaler().fit_transform(info_df[['총대출수']])  # 대출 수 적을수록 높은 점수

# 가중치 1의 최종 점수 (대출일자 70%, 총대출수 30% 반영)
info_df['우선순위_1'] = 0.7 * info_df['대출일자_점수'] + 0.3 * info_df['대출수_점수']

# 2. '중복책권수' 기준 점수 계산 (1인 값은 0으로 설정하고, 2 이상만 정규화)
combined_df['우선순위_2'] = 0

# 2 이상인 값만 비율 기반 변환하여 정규화
mask = combined_df['중복책권수'] > 1
if mask.any():
    scaler = MinMaxScaler()
    combined_df.loc[mask, '우선순위_2'] = scaler.fit_transform(combined_df.loc[mask, ['중복책권수']])

# 3. '출판년도' 기준 점수 계산 (오래된 출판년도일수록 높은 점수)
combined_df['출판년도'] = pd.to_numeric(combined_df['출판년도'], errors='coerce').fillna(0)
combined_df['출판년도_점수'] = 1 - MinMaxScaler().fit_transform(combined_df[['출판년도']])
combined_df['우선순위_3'] = combined_df['출판년도_점수']

# 최종 점수 계산 (가중치: 1 > 2 > 3)
final_df = pd.merge(info_df[['도서ID', '우선순위_1']], combined_df[['도서ID', '중복책권수', '우선순위_2', '우선순위_3']], on='도서ID', how='inner')
final_df['최종우선순위_점수'] = 0.5 * final_df['우선순위_1'] + 0.3 * final_df['우선순위_2'] + 0.2 * final_df['우선순위_3']

# 우선순위 정렬 및 저장
final_df = final_df.sort_values(by='최종우선순위_점수', ascending=False)
final_df.to_csv("최종_우선순위.csv", index=False, encoding='euc-kr')

# 결과 미리보기
print(final_df.head())


변환된 중복책권수 값 확인:
        도서ID  중복책권수
0  SS_126815      1
1  SS_152776      1
2  SS_102154      1
3  SS_208701      1
4  SS_143582      1
5  SS_158302      1
6  SS_155417      1
7  SS_139733      1
8  SS_004435      1
9  SS_152766      1


C:\Users\songbh0304\AppData\Local\Temp\ipykernel_21780\593140052.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  info_df['마지막 대출일자'] = pd.to_datetime(info_df['마지막대출일자'], errors='coerce')
C:\Users\songbh0304\AppData\Local\Temp\ipykernel_21780\593140052.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.         0.         0.         ... 0.00251889 0.00251889 0.00251889]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined_df.loc[mask, '우선순위_2'] = scaler.fit_transform(combined_df.loc[mask, ['중복책권수']])


            도서ID    우선순위_1  중복책권수  우선순위_2  우선순위_3  최종우선순위_점수
16862  SS_131968  0.994329      1     0.0     1.0   0.697164
77795  SS_125795  0.994146      1     0.0     1.0   0.697073
55285  SS_131195  0.973149      1     0.0     1.0   0.686574
96793  SS_180422  0.951577      1     0.0     1.0   0.675788
25455  SS_083207  0.949564      1     0.0     1.0   0.674782


In [28]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# 파일 불러오기
combined_df = pd.read_csv("통합_정보_합침.csv", encoding='euc-kr')

# 중복책권수 데이터 타입 변환
combined_df['중복책권수'] = pd.to_numeric(combined_df['중복책권수'], errors='coerce').fillna(0).astype(int)

# 정규화 전 2 이상인 값만 필터링
mask = combined_df['중복책권수'] >= 2
if not mask.any():  # 중복책권수 >= 2가 없을 경우 모든 값을 0으로 설정
    combined_df['우선순위_2'] = 0
else:
    # 2 이상인 값만 정규화하여 적용
    scaler = MinMaxScaler()
    combined_df.loc[mask, '우선순위_2'] = scaler.fit_transform(combined_df.loc[mask, ['중복책권수']])
    # 2 미만인 값은 0으로 설정
    combined_df.loc[~mask, '우선순위_2'] = 0

# 결과 출력 확인
print("정규화된 우선순위_2 값 분포:")
print(combined_df['우선순위_2'].describe())


정규화된 우선순위_2 값 분포:
count    181058.000000
mean          0.000711
std           0.006823
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: 우선순위_2, dtype: float64
